In [1]:
import grpc
import sys
import inference_pb2_grpc as pb2_grpc
import inference_pb2 as pb2

In [89]:
# Compile proto3 file
# python3 -m grpc_tools.protoc --proto_path=. ./inference.proto --python_out=. --grpc_python_out=.

In [2]:
class TestClient(object):
    """
    Client for testing inference gRPC
    """

    def __init__(self):
        #self.host = '3.139.238.241'
        self.host = 'localhost'
        self.server_port = 5125

        # instantiate a channel
        self.channel = grpc.insecure_channel(
            '{}:{}'.format(self.host, self.server_port))

        # bind the client and the server
        self.stub = pb2_grpc.InferenceStub(self.channel)

    def send(self, tx, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.InferenceParameters(tx=tx, modelHash=modelHash, modelInput = modelInput)
        print(f'{message}')
        return self.stub.RunInference(message)
    
    def pipeline(self, tx, seed, pipelineName, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.PipelineParameters(tx=tx, seed=seed, pipelineName=pipelineName, modelHash=modelHash, modelInput = modelInput)
        return self.stub.RunPipeline(message)

def testLinearModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="LinearTest", modelInput="[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

def testVolatilityModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="Volatility.onnx", modelInput="[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testXGBSpreadModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="XGBSpreadModel", modelInput="[[0.003], [0.005], [0.004056685]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

def testLassoSpreadModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy", modelInput="[[0.003, 0.005, 0.004056685]]")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')
    
def testPipeline():
    client = TestClient()
    result = client.pipeline(tx = "0x123", seed="2", pipelineName="text-generation", modelHash="gpt2-xl", modelInput="Please write me a poem about crypto?")
    print("Querying " + client.host + ":" + str(client.server_port) + "...")
    print(f'{result}')

In [3]:
testVolatilityModel()

tx: "0x123"
modelHash: "Volatility.onnx"
modelInput: "[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]"



_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from Volatility.onnx failed:Load model Volatility.onnx failed. File doesn't exist"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2023-09-29T14:07:35.729537-04:00", grpc_status:2, grpc_message:"Exception calling application: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from Volatility.onnx failed:Load model Volatility.onnx failed. File doesn\'t exist"}"
>

In [4]:
testXGBSpreadModel()

tx: "0x123"
modelHash: "XGBSpreadModel"
modelInput: "[[0.003], [0.005], [0.004056685]]"



_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from XGBSpreadModel failed:Load model XGBSpreadModel failed. File doesn't exist"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Exception calling application: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from XGBSpreadModel failed:Load model XGBSpreadModel failed. File doesn\'t exist", grpc_status:2, created_time:"2023-09-29T14:07:36.496053-04:00"}"
>

In [53]:
testLassoSpreadModel()

tx: "0x123"
modelHash: "QmXQpupTphRTeXJMEz3BCt9YUF6kikcqExxPdcVoL1BBhy"
modelInput: "[[0.003, 0.005, 0.004056685]]"

Querying localhost:5125...
tx: "0x123"
node: "9b438110500ab13fcd0397901332bcac12c5ff4e08f72dd0107acf8d60892a1b34350c7703c52baae1696c2e3229cbadaad4c0d81aefcf115456e3daafb0a519"
value: "0.001360592"



In [106]:
testPipeline()

Querying localhost:5127...
tx: "0x123"
node: "gpt2-xl"
value: "Please write me a poem about crypto?\n\n\nIf you have an internet connection and want to learn about Bitcoin, please write me an email here:\n\n\nIn any case, I am here to help anyone as much as I can!I do"

